In [1]:
from bs4 import BeautifulSoup
import csv
import numpy as np
import pandas as pd
import scraperwiki
from string import ascii_lowercase
import urllib2
import urllib
import time
from selenium   import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
url = 'http://www.fightmetric.com/statistics/fighters'
html = scraperwiki.scrape(url)
soup = BeautifulSoup(html, 'lxml')

In [10]:
table_fighter = soup.find_all('tbody')[0]
rows_fighter = table_fighter.find_all('tr')[1:]

In [11]:
def get_career_stats(url):
    ret = []
    html = scraperwiki.scrape(url)
    soup = BeautifulSoup(html, 'lxml')
    div_stats = soup.find_all('div', {'class': 'b-list__info-box-left clearfix'})[0]
    list_items_stats = div_stats.find_all('li', {'class': 'b-list__box-list-item b-list__box-list-item_type_block'})
    for i in list_items_stats:
        txt = i.text.strip()
        txt = ' '.join(txt.split()).split(': ')[-1]
        if len(txt) > 0:
#             print (txt)
            ret.append(txt)
    return ret

In [12]:
test_url = 'http://www.fightmetric.com/fighter-details/b361180739bed4b0'
get_career_stats(test_url)

[u'1.35', u'30%', u'3.55', u'38%', u'1.07', u'33%', u'66%', u'0.0']

In [13]:
#loop through a single fighter row and capture all the information
def parse_row(row):
    ret = []
    career_stats = []
    
    cols_fighter = row.find_all('td')
    for idx, col in enumerate(cols_fighter):
        links = col.find_all('a')
        num_links = len(links)
        if num_links == 1:
            link = links[0]
            try:
                ret.append(link.contents[0])
                if idx == 0:
                    career_stats = get_career_stats(link['href'])
            except IndexError as e:
                ret.append('n/a')
        elif num_links > 1:
            raise Exception('invalid link parsing')
        else:
    #         print repr(col.contents[0])
            txt = col.contents[0].strip()
            if txt != '':
                if '-' in txt:
                    ret.append('n/a')
                else:
                    ret.append(col.contents[0].strip())
            else:
                ret.append('n/a')
    ret = ret + career_stats
#     print ret
    return ret

In [14]:
header = ['first', 'last', 'nickname', 'height', 'weight', 'reach', 
          'stance', 'w', 'l', 'd', 'belt', 'SLpM', 'Str. Acc.', 'SApM',
         'Str. Def', 'TD Avg', 'TD Acc.', 'TD Def.', 'Sub. Avg.']

In [15]:
res = []
for c in ascii_lowercase[15:16]:
    url = 'http://www.fightmetric.com/statistics/fighters?char=%c&page=all' %c
    print url
    break
    html = scraperwiki.scrape(url)
    soup = BeautifulSoup(html, 'lxml')
    table_fighter = soup.find_all('tbody')[0]
    rows_fighter = table_fighter.find_all('tr')[1:]
    for row in rows_fighter:
        try:
            res.append(parse_row(row))
        except:
            print 'there was an issue with this url: %s' %url

http://www.fightmetric.com/statistics/fighters?char=p&page=all


In [ ]:
df = pd.DataFrame(res, columns=header)
df.to_csv('tmp_data.csv', index=False)

# Pull all of Bisping's fights



In [2]:
#pull all of Bisping's fights
url = 'http://www.fightmetric.com/fighter-details/2b93ebd9f5417ad2'
html = scraperwiki.scrape(url)
soup = BeautifulSoup(html, 'lxml')

In [17]:
table_fighter = soup.find_all('tbody')[0]
rows_fighter = table_fighter.find_all('a', {'class': 'b-link b-link_style_black'})
rows_fighter = [x for x in rows_fighter if 'fighter' in x['href'] and
                'Bisping' not in x.text.strip()]

rows_results = table_fighter.find_all('i', {'class': 'b-flag__text'})

res = [['name', 'result']]
for idx, row in enumerate(rows_fighter):
    fighter = rows_fighter[idx].text.strip()
    result = rows_results[idx].text.strip()
    res.append([fighter, result])

In [19]:
res_np = np.asarray(res)
np.savetxt('bisping_results.csv', res_np, delimiter=',', fmt='%s')

# Pull all fights of all fighters

In [95]:
fighters = np.genfromtxt('data/top10.csv', delimiter=',', dtype=str)
print '%d fighters in the top 10 of each division' %len(fighters)

160 fighters in the top 10 of each division


In [96]:
first,last = fighters[0].split(' ')
print first, last

Demetrious Johnson


In [123]:
#how to get the fight page of each fighter?
d = webdriver.Chrome('/usr/local/lib/chromedriver')
d.implicitly_wait(10) #seconds
d.get('http://www.fightmetric.com/statistics/fighters')

In [126]:
res = []
for fighter in fighters:
    first = fighter.split(' ')[0]
    last = fighter.split(' ')[-1]
    e = d.find_element_by_class_name('b-statistics__search-field')
    e.clear()
    e.send_keys(last)
    e.send_keys(Keys.RETURN)
    url = d.current_url
    html = scraperwiki.scrape(url)
    query_soup = BeautifulSoup(html, 'lxml')
    query_results = query_soup.find_all('a', attrs={'class':'b-link b-link_style_black'})
    for one_query_result in query_results:
        if first in one_query_result.getText():
    #         d.get(one_query_result['href'])
            html = scraperwiki.scrape(one_query_result['href'])
            soup = BeautifulSoup(html, 'lxml')
            table_fighter = soup.find_all('tbody')[0]
            rows_fighter = table_fighter.find_all('a', {'class': 'b-link b-link_style_black'})
            rows_fighter = [x for x in rows_fighter if 'fighter' in x['href'] and
                            last not in x.text.strip()]

            rows_results = table_fighter.find_all('i', {'class': 'b-flag__text'})
            rows_results = [x for x in rows_results if 'next' not in x.text.strip()]
            for idx, row in enumerate(rows_fighter):
                opponent = rows_fighter[idx].text.strip()
                result = rows_results[idx].text.strip()
                res.append([fighter, opponent, result])
            break

In [127]:
np.savetxt('./top15_results.csv',res)

[['Demetrious Johnson', u'Wilson Reis', u'win'],
 ['Demetrious Johnson', u'Tim Elliott', u'win'],
 ['Demetrious Johnson', u'Henry Cejudo', u'win'],
 ['Demetrious Johnson', u'John Dodson', u'win'],
 ['Demetrious Johnson', u'Kyoji Horiguchi', u'win'],
 ['Demetrious Johnson', u'Chris Cariaso', u'win'],
 ['Demetrious Johnson', u'Ali Bagautinov', u'win'],
 ['Demetrious Johnson', u'Joseph Benavidez', u'win'],
 ['Demetrious Johnson', u'John Moraga', u'win'],
 ['Demetrious Johnson', u'John Dodson', u'win'],
 ['Demetrious Johnson', u'Joseph Benavidez', u'win'],
 ['Demetrious Johnson', u'Ian McCall', u'win'],
 ['Demetrious Johnson', u'Ian McCall', u'draw'],
 ['Demetrious Johnson', u'Dominick Cruz', u'loss'],
 ['Demetrious Johnson', u'Miguel Torres', u'win'],
 ['Demetrious Johnson', u'Norifumi Yamamoto', u'win'],
 ['Demetrious Johnson', u'Damacio Page', u'win'],
 ['Demetrious Johnson', u'Nick Pace', u'win'],
 ['Demetrious Johnson', u'Brad Pickett', u'loss'],
 ['Joseph Benavidez', u'Henry Cejudo',

In [125]:
fighters

array(['Demetrious Johnson', 'Joseph Benavidez', 'Henry Cejudo',
       'Ray Borg', 'Sergio Pettis', 'Jussier Formiga', 'Wilson Reis',
       'Brandon Moreno', 'Ben Nguyen', 'Tim Elliott', 'John Moraga',
       'Dustin Ortiz', 'Ian McCall', 'Alexandre Pantoja', 'Louis Smolka',
       'Magomed Bibulatov', 'Cody Garbrandt', 'Dominick Cruz',
       'TJ Dillashaw', 'Jimmie Rivera', 'Raphael Assuncao', 'John Lineker',
       'Bryan Caraway', 'Aljamain Sterling', 'John Dodson',
       'Thomas Almeida', 'Marlon Moraes', 'Eddie Wineland', 'Pedro Munhoz',
       'Rob Font', 'Matthew Lopez', 'Johnny Eduardo', 'Max Holloway',
       'Jose Aldo', 'Frankie Edgar', 'Ricardo Lamas', 'Cub Swanson',
       'Chan Sung Jung', 'Brian Ortega', 'Yair Rodriguez',
       'Jeremy Stephens', 'Darren Elkins', 'Renato Moicano',
       'Dennis Bermudez', 'Dooho Choi', 'Mirsad Bektic', 'Myles Jury',
       'Jason Knight', 'Conor McGregor', 'Khabib Nurmagomedov',
       'Tony Ferguson', 'Eddie Alvarez', 'Edson Barbo